# Récupération du dataset

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Chemin de base vers le dossier 'Data'
base_dir = 'Data'

# Liste pour stocker les dataframes individuels
liste_df = []

# Parcours des dossiers numérotés de '0' à '6'
for i in range(7):
    dossier = str(i)
    chemin_dossier = os.path.join(base_dir, dossier)
    fichier_csv = os.path.join(chemin_dossier, f'df_front_{i}.csv')
    
    try:
        # Lecture du fichier CSV
        df_base = pd.read_csv(fichier_csv)
        
        # Mise à jour du chemin des images pour qu'il pointe vers le bon dossier
        df_base['Image_Path'] = df_base['Image_Path'].apply(lambda x: os.path.join(chemin_dossier, 'images_without_background', x))
        
        # Ajout du dataframe à la liste
        liste_df.append(df_base)
    except FileNotFoundError:
        print(f"Le fichier {fichier_csv} est introuvable. Passage au fichier suivant.")
    except Exception as e:
        print(f"Une erreur est survenue avec le fichier {fichier_csv}: {e}")
        continue

# Concaténation de tous les dataframes en un seul
if liste_df:
    df = pd.concat(liste_df, ignore_index=True)
    print("Tous les datasets ont été combinés avec succès.")
else:
    print("Aucun dataset n'a été chargé.")

Le fichier Data/2/df_front_2.csv est introuvable. Passage au fichier suivant.
Le fichier Data/3/df_front_3.csv est introuvable. Passage au fichier suivant.
Le fichier Data/4/df_front_4.csv est introuvable. Passage au fichier suivant.
Le fichier Data/5/df_front_5.csv est introuvable. Passage au fichier suivant.
Le fichier Data/6/df_front_6.csv est introuvable. Passage au fichier suivant.
Tous les datasets ont été combinés avec succès.


# Traitement des données

In [3]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

/Users/tepe/Desktop/Code/Project/Start-up/Product_Analysis/deep_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Colonnes du DataFrame
IMAGE_COLUMN = 'Image_Path'
LABEL_COLUMN = 'Condition'

# Paramètres du modèle
BATCH_SIZE = 32
IMAGE_SIZE = (380, 380)  # Taille d'entrée pour EfficientNetB4
AUTOTUNE = tf.data.AUTOTUNE
NUM_CLASSES = 5  # Conditions de 0 à 4


# Affichage des premières lignes pour vérification
print("Aperçu du DataFrame :")
print(df.head())

# Analyse de la distribution des classes
print("\nDistribution des classes dans le dataset complet :")
print(df['Condition'].value_counts())

Aperçu du DataFrame :
                                          Image_Path  Condition
0  Data/0/images_without_background/produit_1_ima...          2
1  Data/0/images_without_background/produit_1_ima...          2
2  Data/0/images_without_background/produit_2_ima...          3
3  Data/0/images_without_background/produit_2_ima...          3
4  Data/0/images_without_background/produit_2_ima...          3

Distribution des classes dans le dataset complet :
Condition
3    1855
4     895
2     883
1     217
0     120
Name: count, dtype: int64


In [5]:
# Séparation initiale en entraînement (80%) et temporaire (20%)
train_df, temp_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['Condition'],
    random_state=42
)

# Séparation de l'ensemble temporaire en validation (10%) et test (10%)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df['Condition'],
    random_state=42
)

print(f"\nNombre d'échantillons dans l'ensemble d'entraînement : {len(train_df)}")
print(f"Nombre d'échantillons dans l'ensemble de validation : {len(val_df)}")
print(f"Nombre d'échantillons dans l'ensemble de test : {len(test_df)}")


Nombre d'échantillons dans l'ensemble d'entraînement : 3176
Nombre d'échantillons dans l'ensemble de validation : 397
Nombre d'échantillons dans l'ensemble de test : 397


In [6]:
def load_and_preprocess_image(path, label):
    # Lecture de l'image depuis le fichier
    image = tf.io.read_file(path)
    
    # Décodage de l'image en format RGB
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    
    # Conversion des pixels en flottants entre 0 et 1
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # Redimensionnement avec padding pour maintenir le ratio d'aspect
    image = tf.image.resize_with_pad(image, IMAGE_SIZE[0], IMAGE_SIZE[1])
    
    # Application de la normalisation spécifique à EfficientNetB4
    image = tf.keras.applications.efficientnet.preprocess_input(image * 255.0)
    
    return image, label

In [7]:
# Définition des transformations d'augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
    # Vous pouvez ajouter d'autres couches d'augmentation si nécessaire
])

def augment(image, label):
    image = data_augmentation(image)
    return image, label

In [8]:
def create_dataset(df, shuffle=True, augment_data=False):
    # Extraction des chemins d'images et des labels
    paths = df[IMAGE_COLUMN].values
    labels = df[LABEL_COLUMN].values
    
    # Création d'un dataset TensorFlow à partir des chemins et labels
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    
    # Application de la fonction de prétraitement
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
    
    # Mélange des données si nécessaire
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(df))
    
    # Application de l'augmentation des données si spécifié
    if augment_data:
        dataset = dataset.map(augment, num_parallel_calls=AUTOTUNE)
    
    # Batching et préfetching pour optimiser les performances
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    
    return dataset

In [9]:
# Création des ensembles d'entraînement, de validation et de test
train_dataset = create_dataset(train_df, shuffle=True, augment_data=True)
val_dataset = create_dataset(val_df, shuffle=False, augment_data=False)
test_dataset = create_dataset(test_df, shuffle=False, augment_data=False)

In [10]:
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import EfficientNetB4
from sklearn.utils import class_weight

In [11]:
# Extraction des labels de l'ensemble d'entraînement
train_labels = train_df[LABEL_COLUMN].values

# Calcul des poids de classe
class_weights_array = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Création d'un dictionnaire des poids de classe
class_weights = {i: weight for i, weight in enumerate(class_weights_array)}
print("Poids de classe : ", class_weights)

Poids de classe :  {0: np.float64(6.616666666666666), 1: np.float64(3.650574712643678), 2: np.float64(0.8997167138810198), 3: np.float64(0.4280323450134771), 4: np.float64(0.8871508379888268)}


In [12]:
# Définition de la taille d'entrée
IMAGE_SIZE = (380, 380, 3)  # EfficientNetB4 nécessite des images de taille 380x380 avec 3 canaux de couleur


# Chargement du modèle EfficientNetB4 sans les couches supérieures (top layers)
base_model = EfficientNetB4(
    include_top=False,  # Nous allons ajouter nos propres couches supérieures
    weights='imagenet',  # Utiliser les poids pré-entraînés sur ImageNet
    input_shape=IMAGE_SIZE
)

# Congélation des couches du modèle de base pour le transfert learning initial
base_model.trainable = False

# Vérification des couches congelées
print("Nombre de couches dans le modèle de base : ", len(base_model.layers))
for layer in base_model.layers:
    layer.trainable = False

Nombre de couches dans le modèle de base :  475


In [13]:
# Création du modèle final en ajoutant des couches supérieures personnalisées
model = models.Sequential([
    base_model,  # Le modèle de base EfficientNetB4
    layers.GlobalAveragePooling2D(),  # Réduction de la dimensionnalité
    layers.Dropout(0.5),  # Dropout pour éviter le surapprentissage
    layers.Dense(256, activation='relu'),  # Couche dense avec activation ReLU
    layers.Dropout(0.5),  # Autre couche de Dropout
    layers.Dense(NUM_CLASSES, activation='softmax')  # Couche de sortie avec activation softmax
])

# Affichage de la structure du modèle
#model.summary()

In [14]:
# Compilation du modèle
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),  # Optimiseur Adam avec un taux d'apprentissage initial
    loss='sparse_categorical_crossentropy',  # Fonction de perte adaptée aux labels entiers
    metrics=['accuracy']  # Métrique d'évaluation
)

In [15]:
# Définition des callbacks
checkpoint_cb = callbacks.ModelCheckpoint(
    'best_model.keras',  # Chemin où enregistrer le meilleur modèle
    save_best_only=True,  # Enregistrer uniquement le modèle avec la meilleure performance sur la validation
    monitor='val_accuracy',  # Surveiller la précision sur l'ensemble de validation
    mode='max'
)

earlystop_cb = callbacks.EarlyStopping(
    monitor='val_accuracy',  # Surveiller la précision sur l'ensemble de validation
    patience=10,  # Arrêter l'entraînement après 10 époques sans amélioration
    restore_best_weights=True  # Restaurer les poids du meilleur modèle
)

reduce_lr_cb = callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',  # Surveiller la précision sur l'ensemble de validation
    factor=0.2,  # Facteur de réduction du taux d'apprentissage
    patience=5,  # Patience avant de réduire le taux d'apprentissage
    min_lr=1e-6  # Taux d'apprentissage minimum
)

In [16]:
# Nombre d'époques initial
initial_epochs = 30

# Entraînement du modèle
history = model.fit(
    train_dataset,  # Ensemble d'entraînement
    epochs=initial_epochs,  # Nombre d'époques
    validation_data=val_dataset,  # Ensemble de validation
    class_weight=class_weights,  # Poids de classe pour gérer le déséquilibre
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb]  # Callbacks définis précédemment
)

Epoch 1/30


2024-12-06 13:39:11.314285: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 2483 of 3176
2024-12-06 13:39:14.248223: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


100/100 ━━━━━━━━━━━━━━━━━━━━ 402s 3s/step - accuracy: 0.2072 - loss: 1.7748 - val_accuracy: 0.2292 - val_loss: 1.5590 - learning_rate: 0.0010
Epoch 2/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 239s 2s/step - accuracy: 0.2155 - loss: 1.6438 - val_accuracy: 0.1436 - val_loss: 1.5906 - learning_rate: 0.0010
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 241s 2s/step - accuracy: 0.2618 - loss: 1.5490 - val_accuracy: 0.1562 - val_loss: 1.6046 - learning_rate: 0.0010
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 255s 3s/step - accuracy: 0.2510 - loss: 1.4792 - val_accuracy: 0.2191 - val_loss: 1.5372 - learning_rate: 0.0010
Epoch 5/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - accuracy: 0.2821 - loss: 1.4584 - val_accuracy: 0.1914 - val_loss: 1.6010 - learning_rate: 0.0010
Epoch 6/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 239s 2s/step - accuracy: 0.2915 - loss: 1.4202 - val_accuracy: 0.1965 - val_loss: 1.6049 - learning_rate: 0.0010
Epoch 7/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 257s 3s/step - accuracy: 0.2699 - loss: 1.4705 - val_

KeyboardInterrupt: 

In [ ]:
# Définir combien de couches du modèle de base seront dégelées
total_layers = len(base_model.layers)
fine_tune_at = total_layers - 30  # Dégeler les dernières 30 couches

# Congélation partielle du modèle de base
base_model.trainable = True

# Congélation des couches jusqu'à fine_tune_at
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompilation du modèle avec un taux d'apprentissage plus bas pour le fine-tuning
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Taux d'apprentissage réduit
    loss='sparse_categorical_crossentropy',  # Même fonction de perte
    metrics=['accuracy']  # Même métrique
)

# Nombre d'époques pour le fine-tuning
fine_tune_epochs = 20
total_epochs = initial_epochs + fine_tune_epochs

# Entraînement avec fine-tuning
history_fine = model.fit(
    train_dataset,  # Ensemble d'entraînement
    epochs=total_epochs,  # Nombre total d'époques
    initial_epoch=history.epoch[-1],  # Commencer après les époques initiales
    validation_data=val_dataset,  # Ensemble de validation
    class_weight=class_weights,  # Poids de classe
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb]  # Callbacks
)